In [2]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Utah 2016

## VEST Documentation

### Data Sources

Election results from OpenElections.

Shapefile from the Utah Geographic Reference Center (https://gis.utah.gov/data/political/voter-precincts/).


### Election Processing

The shapefile from the UGRC is of subprecincts, breaking down precincts in cases of district splits. In some cases, results are reported at the subprecinct level, in most cases, they weren't, so merging was done where necessary.


### Candidates

G16PRERTRU - Donald J. Trump (Republican Party) 
G16PREDCLI - Hillary Rodham Clinton (Democratic Party)
G16PRELJOH - Gary Johnson (Libertarian Party)
G16PREISTE - Jill Stein (Unaffiliated)
G16PREIMCM - Evan McMullen (Unaffiliated)
G16PREOOTH - Other candidates on ballot

G16USSRLEE - Mike Lee (Republican Party)
G16USSDSNO - Misty K. Snow (Democratic Party)
G16USSAFON - Stoney Fonua (Independent American Party)
G16USSIBAR - Bill Barron (Unaffiliated)

G16GOVRHER - Gary R. Herbert (Republican Party)
G16GOVDWEI - Mike Weinholtz (Democratic Party)
G16GOVLKAM - Brian E. Kamerath (Libertarian Party)
G16GOVASCH - Superdell Schanze (Independent American Party)

G16ATGRREY - Sean D. Reyes (Republican Party)
G16ATGDHAR - Jon V. Harper (Democratic Party)
G16ATGLMCC - W. Andrew McCullogh (Libertarian Party)
G16ATGAISB - Michael W. IsBell (Independent American Party)

G16AUDRDOU - John Rougall (Republican Party)
G16AUDDMIT - Mike Mitchell (Democratic Party)
G16AUDAGRE - Jared Green (Independent American Party)

G16TRERDAM - David Damschen (Republican Party)
G16TREDHAN - Neil A. Hansen (Democratic Party)
G16TRECPRO - Richard Proctor (Constitution Party)


In [3]:
#Load UT election results at the precinct level
ut_election_results = pd.read_csv('raw-from-source/20161108__ut__general__precinct.csv')

#Load the final VEST file
ut_vest = gp.read_file("raw-from-source/ut_2016/ut_2016.shp")

#Load the UT 2016 VEST file
ut_vest_2016 = gp.read_file("raw-from-source/ut_2016/ut_2016.shp")

#Load UT election results at the county level
ut_vest_county = pd.read_csv('raw-from-source/20161108__ut__general__county.csv')

#Load in UGRS shapefile
ut_geo = gp.read_file('raw-from-source/Utah_Vista_Ballot_Areas/VistaBallotAreas.shp')

In [4]:
file_list = os.listdir("./raw-from-source/counties")
li = []
for file in file_list:
    if "general" in file:
        #print(file)
        file_ref = "./raw-from-source/counties/"+file
        file_prev = pd.read_csv(file_ref)
        if 'precincts' in file_prev.columns:
            file_prev['precinct'] = file_prev['precincts']
     
        file_prev = file_prev[['county', 'precinct', 'office', 'district', 'party', 'candidate','votes']]
    print(file_prev.shape)
    li.append(file_prev)
county_df = pd.concat(li, axis=0, ignore_index=True)
display(county_df)

(1017, 7)
(10488, 7)
(624, 7)
(432, 7)
(11297, 7)
(853, 7)
(646, 7)
(496, 7)
(216, 7)
(396, 7)
(6004, 7)
(720, 7)
(351, 7)
(4013, 7)
(1408, 7)
(608, 7)
(936, 7)
(1668, 7)
(3702, 7)
(648, 7)
(1908, 7)
(1310, 7)
(185, 7)
(700, 7)
(1224, 7)
(1224, 7)
(108, 7)
(238, 7)
(2275, 7)
(42686, 7)


,county,precinct,office,district,party,candidate,votes
0,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Rocky De La Fuente,0.0
1,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Jill Stein,2.0
2,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Alyson Kennedy,0.0
3,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Libertarian,Gary Johnson,12.0
4,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Independent American,Rocky Giordani,0.0
...,...,...,...,...,...,...,...
98376,Salt Lake,SJD023,State House,52.0,Republican,John Knotwell,386.0
98377,Salt Lake,SJD025,State House,52.0,Republican,John Knotwell,377.0
98378,Salt Lake,SJD029,State House,52.0,Republican,John Knotwell,429.0
98379,Salt Lake,SWT001,State House,52.0,Republican,John Knotwell,266.0


In [5]:
list_of_races = ['President', 'U.S. Senate', 'Governor', 'Attorney General', 'State Auditor', 'State Treasurer']
clean_df = county_df[county_df['office'].isin(list_of_races)]

In [6]:
clean_df['candidate_office'] = clean_df['candidate'] + '/' + clean_df['office']
display(clean_df)

<ipython-input-6-27ffffdc4b6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['candidate_office'] = clean_df['candidate'] + '/' + clean_df['office']


,county,precinct,office,district,party,candidate,votes,candidate_office
0,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Rocky De La Fuente,0.0,Rocky De La Fuente/President
1,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Jill Stein,2.0,Jill Stein/President
2,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Alyson Kennedy,0.0,Alyson Kennedy/President
3,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Libertarian,Gary Johnson,12.0,Gary Johnson/President
4,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Independent American,Rocky Giordani,0.0,Rocky Giordani/President
...,...,...,...,...,...,...,...,...
96109,Salt Lake,WVC076,State Treasurer,NaN,Republican,David Damschen,52.0,David Damschen/State Treasurer
96110,Salt Lake,WVC077,State Treasurer,NaN,Republican,David Damschen,106.0,David Damschen/State Treasurer
96111,Salt Lake,WVC078,State Treasurer,NaN,Republican,David Damschen,133.0,David Damschen/State Treasurer
96112,Salt Lake,WVC079,State Treasurer,NaN,Republican,David Damschen,107.0,David Damschen/State Treasurer


In [7]:
cand_list = clean_df.candidate.unique()
cand_list = [x for x in cand_list if pd.isnull(x) == False and x != 'nan']

cand_office_list = clean_df.candidate_office.unique()
cand_office_list = [x for x in cand_office_list if pd.isnull(x) == False and x != 'nan']

for cand in cand_list:
    #print(cand)
    pass


print('number of rows in original dataframe is:', len(clean_df))

null_precincts = (clean_df[clean_df['candidate_office'].isnull()]) #precincts where candidate is NaN
non_null_df = clean_df.drop(null_precincts.index)
print(len(null_precincts), len(non_null_df))
party_precincts = null_precincts[null_precincts['party'].isin(cand_list)]
party_precincts['candidate_office'] = party_precincts['party'] + '/' + party_precincts['office']
cand_office_df = pd.concat([non_null_df, party_precincts])
display(cand_office_df)
nan_precincts = null_precincts.drop(party_precincts.index)

print(len(cand_office_df), len(nan_precincts))
registered_voters = (nan_precincts[nan_precincts['party'].isnull()])
non_nan = nan_precincts.drop(registered_voters.index)
display(cand_office_df)

number of rows in original dataframe is: 84594
42 84552


,county,precinct,office,district,party,candidate,votes,candidate_office
0,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Rocky De La Fuente,0.0,Rocky De La Fuente/President
1,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Jill Stein,2.0,Jill Stein/President
2,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Alyson Kennedy,0.0,Alyson Kennedy/President
3,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Libertarian,Gary Johnson,12.0,Gary Johnson/President
4,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Independent American,Rocky Giordani,0.0,Rocky Giordani/President
...,...,...,...,...,...,...,...,...
53387,Wayne,Hanksville,President,NaN,Jill Stein,NaN,0.0,Jill Stein/President
53388,Wayne,Hanksville,President,NaN,Alyson Kennedy,NaN,0.0,Alyson Kennedy/President
53394,Wayne,Hanksville,President,NaN,Monica Moorehead,NaN,0.0,Monica Moorehead/President
53395,Wayne,Hanksville,President,NaN,Evan McMullin,NaN,20.0,Evan McMullin/President


84594 0


,county,precinct,office,district,party,candidate,votes,candidate_office
0,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Rocky De La Fuente,0.0,Rocky De La Fuente/President
1,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Jill Stein,2.0,Jill Stein/President
2,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Alyson Kennedy,0.0,Alyson Kennedy/President
3,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Libertarian,Gary Johnson,12.0,Gary Johnson/President
4,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Independent American,Rocky Giordani,0.0,Rocky Giordani/President
...,...,...,...,...,...,...,...,...
53387,Wayne,Hanksville,President,NaN,Jill Stein,NaN,0.0,Jill Stein/President
53388,Wayne,Hanksville,President,NaN,Alyson Kennedy,NaN,0.0,Alyson Kennedy/President
53394,Wayne,Hanksville,President,NaN,Monica Moorehead,NaN,0.0,Monica Moorehead/President
53395,Wayne,Hanksville,President,NaN,Evan McMullin,NaN,20.0,Evan McMullin/President


In [11]:
cand_dict = {
    
    'Donald Trump/President': 'G16PRERTRU', 
    'Republican/President': 'G16PRERTRU', 

    'Hillary Clinton/President': 'G16PREDCLI', 
    'Democratic/President': 'G16PREDCLI', 

    'Gary Johnson/President': 'G16PRELJOH',
    'Libertarian/President': 'G16PRELJOH', 
    
    'Jill Stein/President': 'G16PREISTE', 
    
    'Evan McMullin/President': 'G16PREIMCM', 


    'Rocky De La Fuente/President': 'G16PREOOTH', 
    'Alyson Kennedy/President': 'G16PREOOTH', 
    'Rocky Giordani/President': 'G16PREOOTH', 
    'Darrell Castle/President': 'G16PREOOTH', 
    'Monica Moorehead/President': 'G16PREOOTH', 
    'Write In/President': 'G16PREOOTH', 
    'Fuente/President': 'G16PREOOTH', 
    'Giordani/President': 'G16PREOOTH', 
    'Stephen Park/President': 'G16PREOOTH', 
    'Mike Smith/President': 'G16PREOOTH', 
    'Laurence Kotlikoff/President': 'G16PREOOTH', 
    'Hoefling/President': 'G16PREOOTH', 
    'Limbaugh/President': 'G16PREOOTH', 
    'Dustin Baird/President': 'G16PREOOTH',
    'Andrew Basiago/President': 'G16PREOOTH', 
    'Emidio Soltyisk/President': 'G16PREOOTH', 
    'Tony Valdivia/President': 'G16PREOOTH', 
    'Fox/President': 'G16PREOOTH',
    'Sheila Samm Tittle/President': 'G16PREOOTH', 
    'Buchnan/President': 'G16PREOOTH', 
    'Marshall Schoenke/President': 'G16PREOOTH', 
    'Janet Reid/President': 'G16PREOOTH', 
    'Burton/President': 'G16PREOOTH', 
    'Write-In/President': 'G16PREOOTH',
    'Giordani,Rocky/President': 'G16PREOOTH',
    'Roque De La Fuente/President': 'G16PREOOTH', 
    'Darrell L. Castle/President': 'G16PREOOTH', 
    'Tony Valdiva/President': 'G16PREOOTH',
    'Write-ins/President': 'G16PREOOTH',
    "'Rocky' De La Fuente/President": 'G16PREOOTH', 
    'Giordani/Anderson/President': 'G16PREOOTH', 
    'Independent American/President': 'G16PREOOTH', 
    'Constitution/President': 'G16PREOOTH', 
    'Stephen Paul Parks/President': 'G16PREOOTH', 
    'Tom Hoefling/President': 'G16PREOOTH', 
    'Emidio Soltysik/President': 'G16PREOOTH', 
    'Jamin Burton/President': 'G16PREOOTH', 
    'Robert Buchanan/President': 'G16PREOOTH', 
    'Cherunda Fox/President': 'G16PREOOTH', 
    'David Limbaugh/President': 'G16PREOOTH', 
    
    
    
    'Mike Lee/U.S. Senate': 'G16USSRLEE',
    'Republican/U.S. Senate': 'G16USSRLEE', 
    
    'Misty Snow/U.S. Senate': 'G16USSDSNO', 
    'Misty K. Snow/U.S. Senate': 'G16USSDSNO',
    'Democratic/U.S. Senate': 'G16USSDSNO', 

    'Stoney Fonua/U.S. Senate': 'G16USSAFON', 
    'Independent American/U.S. Senate': 'G16USSAFON', 

    'Bill Barron/U.S. Senate': 'G16USSIBAR', 

    
    
    'Gary Herbert/Governor': 'G16GOVRHER', 
    'Gary R. Herbert/Governor': 'G16GOVRHER', 
    'Gray Herbert/Governor': 'G16GOVRHER', 
    'Republican/Governor': 'G16GOVRHER', 
    'Herbert/Governor': 'G16GOVRHER', 

    'Mike Weinholtz/Governor': 'G16GOVDWEI', 
    'Democratic/Governor': 'G16GOVDWEI', 

    'Brian Kamerath/Governor': 'G16GOVLKAM', 
    'Brian E. Kamerath/Governor': 'G16GOVLKAM',
    'Libertarian/Governor': 'G16GOVLKAM', 

    'Dell Schanze/Governor': 'G16GOVASCH', 
    'Superdell Schanze/Governor': 'G16GOVASCH', 
    'Independent American/Governor': 'G16GOVASCH', 
    
    'L.S. Brown/Governor': None, 
    'Write In/Governor': None, 
    'Write-In/Governor': None, 
    'Write-ins/Governor': None, 

    
    
    'Sean Reyes/Attorney General': 'G16ATGRREY', 
    'Sean D. Reyes/Attorney General': 'G16ATGRREY',
    'Republican/Attorney General': 'G16ATGRREY', 

    'Jon V. Harper/Attorney General': 'G16ATGDHAR', 
    'Jon Harper/Attorney General': 'G16ATGDHAR', 
    'Democratic/Attorney General': 'G16ATGDHAR', 

    'W. Andrew McCullough/Attorney General': 'G16ATGLMCC', 
    'Libertarian/Attorney General': 'G16ATGLMCC', 
    
    'Michael Isbell/Attorney General': 'G16ATGAISB', 
    'Michael W. Isbell/Attorney General': 'G16ATGAISB', 
    'Independent American/Attorney General': 'G16ATGAISB', 

    
    
    'John Dougall/State Auditor': 'G16AUDRDOU', 
    'Republican/State Auditor': 'G16AUDRDOU', 
    
    'Mike Mitchell/State Auditor': 'G16AUDDMIT', 
    'Democratic/State Auditor': 'G16AUDDMIT', 

    'Jared Green/State Auditor': 'G16AUDAGRE', 
    'Independent American/State Auditor': 'G16AUDAGRE', 
    
    'Green/State Auditor': None, 

    
    
    'David Damschen/State Treasurer': 'G16TRERDAM', 
    'Davad Damschen/State Treasurer': 'G16TRERDAM',
    'Republican/State Treasurer': 'G16TRERDAM',
    
    'Neil A. Hansen/State Treasurer': 'G16TREDHAN',
    'Neil Hansen/State Treasurer': 'G16TREDHAN', 
    'Hansen/State Treasurer': 'G16TREDHAN',
    'Democratic/State Treasurer': 'G16TREDHAN', 

    'Richard Proctor/State Treasurer': 'G16TRECPRO', 
    'Constitution/State Treasurer': 'G16TRECPRO'
}

other_dict = {
    
    'Collin Simonsen/U.S. House': None, 
    'Doug Owens/U.S. House': None, 
    'Mia Love/U.S. House': None, 
    'Chris Stewart/U.S. House': None, 
    'Charlene Albarran/U.S. House': None, 
    'Paul McCollaum/U.S. House': None,
    'Rob Bishop/U.S. House': None, 
    'Craig Bowden/U.S. House': None, 
    'Chadwick Fairbanks/U.S. House': None, 
    'Peter Clemens/U.S. House': None,
    'Stephen Tryon/U.S. House': None, 
    'Jason Chaffetz/U.S. House': None,
    'Chadwick H. Fairbanks/U.S. House': None, 
    'Peter C. Clemens/U.S. House': None, 
    'Republican/U.S. House': None, 
    'Democratic/U.S. House': None, 
    'Constitution/U.S. House': None, 

    
    'Ralph Okerlund/State Senate': None,
    'Buxton/State Senate': None,
    'Alan Yorgason/State Senate': None,
    'Hartwick/State Senate': None, 
    'Todd Weiler/State Senate': None,
    'Deidre Henderson/State Senate': None, 
    'Andrew Apsley/State Senate': None, 
    'Jake Anderegg/State Senate': None,
    'Write-In/State Senate': None, 
    'Dan Hemmert/State Senate': None, 
    'Joe Buchman/State Senate': None, 
    'Curt Crosby/State Senate': None,
    'Curt Bramble/State Senate': None, 
    'Jason Christensen/State Senate': None, 
    'David Hinkins/State Senate': None,
    'Heidi Redd/State Senate': None,
    'Deana Froerer/State Senate': None, 
    'Allen Christensen/State Senate': None, 
    'Gregg Buxton/State Senate': None,
    'Lyle W. Hillyard/State Senate': None,
    'Dorothy Engelman/State Senate': None, 
    'Lyle Hillyard/State Senate': None, 
    'Republican/State Senate': None, 
    'Don Ipson/State Senate': None, 
    'Luz Escamilla/State Senate': None, 
    'Fred Johnson/State Senate': None, 
    'Wayne Harper/State Senate': None, 
    'Jim Dexter/State Senate': None, 
    'Celina Milner/State Senate': None, 
    'Brian Shiozawa/State Senate': None, 
    'Ash Anderson/State Senate': None, 
    'Dan Paget/State Senate': None, 
    'Lincoln Fillmore/State Senate': None, 
    'David (Dave) Lifferth/State Senate': None, 
    'Steve Hartwick/State Senate': None, 
 
    
    'Derrin Owens/State House': None,  
    'Carl Albrecht/State House': None, 
    'William Groff/State House': None,
    'Kelly Miles/State House': None, 
    'Amy Morgan/State House': None, 
    'Mike Schultz/State House': None, 
    'Bob Buckles/State House': None, 
    'Paul Ray/State House': None, 
    'Karianne Lisonbee/State House': None, 
    'Rich Miller/State House': None, 
    'Brad Wilson/State House': None, 
    'Steve Handy/State House': None, 
    'Tiffany Kopp/State House': None, 
    'Brent Zimmerman/State House': None, 
    'Christine Stenquist/State House': None, 
    'Stewart Barlow/State House': None, 
    'Timothy Hawkes/State House': None, 
    'Kurt Weiland/State House': None, 
    'Raymond Ward/State House': None, 
    'Becky Edwards/State House': None, 
    'Jon Marsh/State House': None, 
    'Kirk Pearson/State House': None, 
    'Merrill Nelson/State House': None, 
    'Cindee Beard/State House': None,  
    'Mike Noel/State House': None, 
    'Ty Markham/State House': None, 
    'Jefferson Moss/State House': None, 
    'Aaron Davis/State House': None, 
    'Donna Gibbons/State House': None, 
    'Cory Maloy/State House': None, 
    'Mike Kennedy/State House': None, 
    'Keven Stratton/State House': None, 
    'Kay Christofferson/State House': None, 
    'Brian Greene/State House': None, 
    'Val Peterson/State House': None, 
    'Rachel Nelson/State House': None, 
    'Bradley Daw/State House': None, 
    'B. Swallow-Fenton/State House': None, 
    'Tommy Williams/State House': None, 
    'Keith Grover/State House': None, 
    'Dean Sanpei/State House': None, 
    'Nathan Smith Jones/State House': None,
    'Norm Thuston/State House': None, 
    'Francis Gibson/State House': None, 
    'Mike McKell/State House': None, 
    'Marc Roberts/State House': None, 
    'Brad King/State House': None, 
    'Christine Watkins/State House': None,
    'Floyd Handley/State House': None, 
    'Justin Fawson/State House': None, 
    'Matthew Frandsen/State House': None, 
    'Gage Froerer/State House': None,
    'Jeremy Peterson/State House': None, 
    'Kathie Darby/State House': None, 
    'Dixon Pitcher/State House': None, 
    'Jesus Garcia/State House': None, 
    'Derryck Gordon/State House': None, 
    'Lee Perry/State House': None, 
    'Angela Urrea/State House': None,
    'Sherry Phipps/State House': None,
    'Scott Sandall/State House': None,
    'Val K. Potter/State House': None,
    'Edward Redd/State House': None, 
    'David L. Clark/State House': None, 
    'R. Curt Webb/State House': None, 
    'Chuck Goode/State House': None, 
    'Brad Last/State House': None, 
    'John Westwood/State House': None, 
    'Cole Capener/State House': None, 
    'Logan Wilde/State House': None,   
    'Wayne Stevens/State House': None, 
    'Scott Chew/State House': None, 
    'Brian King/State House': None, 
    'Rudi Kohler/State House': None, 
    'Tim Quinn/State House': None, 
    'Jon Stanard/State House': None, 
    'V Lowry Snow/State House': None, 
    'Walt Brooks/State House': None, 
    'Republican/State House': None, 
    'Democratic/State House': None, 
    'Rick Pollock/State House': None, 
    'Douglas Sagers/State House': None, 
    'Susan Duckworth/State House': None, 
    'Jamie White/State House': None, 
    'Scott Hawkins/State House': None, 
    'Sandra Hollins/State House': None, 
    'Rebecca Chavez Houck/State House': None, 
    'Joel Briscoe/State House': None, 
    'Angela Romero/State House': None, 
    'Frank Bedolla/State House': None, 
    'Mike Winder/State House': None, 
    'Sophia Dicaro/State House': None, 
    'Elizabeth Weight/State House': None, 
    'Suzanne Harrison/State House': None, 
    'Lavar Christensen/State House': None, 
    'Craig Hall/State House': None, 
    'Peter Tomala/State House': None, 
    'Karen Kwan/State House': None, 
    'Macade Jensen/State House': None, 
    'Patricia Phaklides/State House': None, 
    'Chelsea Travis/State House': None, 
    'Mark Wheatley/State House': None, 
    'Patrice Arent/State House': None, 
    'Kris Kimball/State House': None, 
    'Carol Spackman/State House': None, 
    'Charles Henderson/State House': None, 
    'Eric Hutchings/State House': None, 
    'Jim Dunnigan/State House': None, 
    'Paul Schulte/State House': None, 
    'Lynn Hemingway/State House': None, 
    'Joseph Breault/State House': None, 
    'Chad Harrington/State House': None, 
    'Dan McCay/State House': None, 
    'Kim Coleman/State House': None, 
    'Edgar Harwood/State House': None, 
    'Adam Gardiner/State House': None, 
    'Christine Passey/State House': None, 
    'Bruce Cutler/State House': None, 
    'Steve Eliason/State House': None, 
    'Nikki Cunard/State House': None, 
    'Marie Poulson/State House': None, 
    'Lee Anne Walker/State House': None, 
    'Ken Ivory/State House': None, 
    'John Rendell/State House': None, 
    'Robert Spendlove/State House': None, 
    'Zach Robinson/State House': None, 
    'Susan Pulsipher/State House': None, 
    'Patty Rich/State House': None, 
    'Greg Hughes/State House': None, 
    'Kyle Waters/State House': None, 
    'Gordon Jones/State House': None, 
    'Write-In/State House': None, 
    'Garr Smith/State House': None, 
    'John Knotwell/State House': None
}

In [12]:
display(cand_office_df)

,county,precinct,office,district,party,candidate,votes,candidate_office
0,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Rocky De La Fuente,0.0,Rocky De La Fuente/President
1,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Jill Stein,2.0,Jill Stein/President
2,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Alyson Kennedy,0.0,Alyson Kennedy/President
3,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Libertarian,Gary Johnson,12.0,Gary Johnson/President
4,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Independent American,Rocky Giordani,0.0,Rocky Giordani/President
...,...,...,...,...,...,...,...,...
53387,Wayne,Hanksville,President,NaN,Jill Stein,NaN,0.0,Jill Stein/President
53388,Wayne,Hanksville,President,NaN,Alyson Kennedy,NaN,0.0,Alyson Kennedy/President
53394,Wayne,Hanksville,President,NaN,Monica Moorehead,NaN,0.0,Monica Moorehead/President
53395,Wayne,Hanksville,President,NaN,Evan McMullin,NaN,20.0,Evan McMullin/President


In [14]:
cand_office_df['CandidateID'] = cand_office_df['candidate_office'].map(cand_dict)

In [15]:
display(cand_office_df)

,county,precinct,office,district,party,candidate,votes,candidate_office,CandidateID
0,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Rocky De La Fuente,0.0,Rocky De La Fuente/President,G16PREOOTH
1,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Jill Stein,2.0,Jill Stein/President,G16PREISTE
2,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Alyson Kennedy,0.0,Alyson Kennedy/President,G16PREOOTH
3,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Libertarian,Gary Johnson,12.0,Gary Johnson/President,G16PRELJOH
4,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Independent American,Rocky Giordani,0.0,Rocky Giordani/President,G16PREOOTH
...,...,...,...,...,...,...,...,...,...
53387,Wayne,Hanksville,President,NaN,Jill Stein,NaN,0.0,Jill Stein/President,G16PREISTE
53388,Wayne,Hanksville,President,NaN,Alyson Kennedy,NaN,0.0,Alyson Kennedy/President,G16PREOOTH
53394,Wayne,Hanksville,President,NaN,Monica Moorehead,NaN,0.0,Monica Moorehead/President,G16PREOOTH
53395,Wayne,Hanksville,President,NaN,Evan McMullin,NaN,20.0,Evan McMullin/President,G16PREIMCM


In [16]:
race_df = cand_office_df[~cand_office_df['CandidateID'].isnull()]

In [17]:
display(race_df)

,county,precinct,office,district,party,candidate,votes,candidate_office,CandidateID
0,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Rocky De La Fuente,0.0,Rocky De La Fuente/President,G16PREOOTH
1,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Jill Stein,2.0,Jill Stein/President,G16PREISTE
2,San Pete,FAIRVIEW MUNICIPAL,President,NaN,NaN,Alyson Kennedy,0.0,Alyson Kennedy/President,G16PREOOTH
3,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Libertarian,Gary Johnson,12.0,Gary Johnson/President,G16PRELJOH
4,San Pete,FAIRVIEW MUNICIPAL,President,NaN,Independent American,Rocky Giordani,0.0,Rocky Giordani/President,G16PREOOTH
...,...,...,...,...,...,...,...,...,...
53387,Wayne,Hanksville,President,NaN,Jill Stein,NaN,0.0,Jill Stein/President,G16PREISTE
53388,Wayne,Hanksville,President,NaN,Alyson Kennedy,NaN,0.0,Alyson Kennedy/President,G16PREOOTH
53394,Wayne,Hanksville,President,NaN,Monica Moorehead,NaN,0.0,Monica Moorehead/President,G16PREOOTH
53395,Wayne,Hanksville,President,NaN,Evan McMullin,NaN,20.0,Evan McMullin/President,G16PREIMCM


In [22]:
dropped_votes = [
    'L.S. Brown/Governor',
    'Write In/Governor',
    'Write-In/Governor',
    'Write-ins/Governor',
    'Green/State Auditor'
]
dropped_df = cand_office_df[cand_office_df['candidate_office'].isin(dropped_votes)]
print('Length of total dataframe: ', len(cand_office_df))
print('Length of cleaned df plus candidates not recorded by VEST: ', (len(race_df) + len(dropped_df)))

Length of total dataframe:  84594
Length of cleaned df plus candidates not recorded by VEST:  84594


In [43]:
#county cast
county_to_num = {
    'Beaver': 1, 
    'Box Elder': 2, 
    'Cache': 3, 
    'Carbon': 4, 
    'Daggett': 5, 
    'Davis': 6, 
    'Duchesne': 7,
    'Emery': 8, 
    'Garfield': 9, 
    'Grand': 10, 
    'Iron': 11, 
    'Juab': 12, 
    'Kane': 13, 
    'Millard': 14, 
    'Morgan': 15, 
    'Piute': 16, 
    'Rich': 17, 
    'Salt Lake': 18,
    'San Juan': 19, 
    'Sanpete': 20, 
    'San Pete': 20, 
    'Sevier': 21, 
    'Summit': 22, 
    'Tooele': 23,
    'Uintah': 24, 
    'Utah': 25,
    'Wasatch': 26,
    'Washington': 27,
    'Wayne': 28,
    'Weber': 29
}

,county,precinct,office,district,party,candidate,votes,candidate_office,CandidateID,CountyID,mergeid


In [47]:
cand_office_df['CountyID'] = cand_office_df['county'].map(county_to_num).astype(int)

In [48]:
cand_office_df['mergeid'] = cand_office_df['CountyID'].astype(str)+'-'+cand_office_df['precinct']

In [83]:
county7 = (cand_office_df[cand_office_df['county'] == 'Duchesne'])
display(county7[county7['CandidateID'] == 'G16AUDRDOU'])

,county,precinct,office,district,party,candidate,votes,candidate_office,CandidateID,CountyID,mergeid
51578,Duchesne,AL11,State Auditor,NaN,Republican,John Dougall,249.0,John Dougall/State Auditor,G16AUDRDOU,7,7-AL11
51581,Duchesne,AL12,State Auditor,NaN,Republican,John Dougall,58.0,John Dougall/State Auditor,G16AUDRDOU,7,7-AL12
51584,Duchesne,AR11,State Auditor,NaN,Republican,John Dougall,105.0,John Dougall/State Auditor,G16AUDRDOU,7,7-AR11
51587,Duchesne,BB11,State Auditor,NaN,Republican,John Dougall,227.0,John Dougall/State Auditor,G16AUDRDOU,7,7-BB11
51590,Duchesne,BB21,State Auditor,NaN,Republican,John Dougall,101.0,John Dougall/State Auditor,G16AUDRDOU,7,7-BB21
...,...,...,...,...,...,...,...,...,...,...,...
52889,Duchesne,RO82,State Auditor,NaN,Republican,John Dougall,72.0,John Dougall/State Auditor,G16AUDRDOU,7,7-RO82
52892,Duchesne,RO91,State Auditor,NaN,Republican,John Dougall,3.0,John Dougall/State Auditor,G16AUDRDOU,7,7-RO91
52895,Duchesne,RO92,State Auditor,NaN,Republican,John Dougall,9.0,John Dougall/State Auditor,G16AUDRDOU,7,7-RO92
52898,Duchesne,TA11,State Auditor,NaN,Republican,John Dougall,175.0,John Dougall/State Auditor,G16AUDRDOU,7,7-TA11


In [82]:
pivoted_df = pd.pivot_table(cand_office_df,values=["votes"],index=["mergeid"],columns=["CandidateID"],aggfunc=sum)
pivoted_df=pivoted_df.fillna(0)
pivoted_df.reset_index(drop=False,inplace=True)
pivoted_df.columns = pivoted_df.columns.droplevel(0)

pivoted_df.columns = [
    'joinid', 'G16ATGAISB', 'G16ATGDHAR', 'G16ATGLMCC',
    'G16ATGRREY', 'G16AUDAGRE', 'G16AUDDMIT', 'G16AUDRDOU',
    'G16GOVASCH', 'G16GOVDWEI', 'G16GOVLKAM', 'G16GOVRHER',
    'G16PREDCLI', 'G16PREIMCM', 'G16PREISTE', 'G16PRELJOH',
    'G16PREOOTH', 'G16PRERTRU', 'G16TRECPRO', 'G16TREDHAN',
    'G16TRERDAM', 'G16USSAFON', 'G16USSDSNO', 'G16USSIBAR', 
    'G16USSRLEE']

display(pivoted_df)

display(pivoted_df[pivoted_df['joinid'] == '7-AL11'])

,joinid,G16ATGAISB,G16ATGDHAR,G16ATGLMCC,G16ATGRREY,G16AUDAGRE,G16AUDDMIT,G16AUDRDOU,G16GOVASCH,G16GOVDWEI,G16GOVLKAM,G16GOVRHER,G16PREDCLI,G16PREIMCM,G16PREISTE,G16PRELJOH,G16PREOOTH,G16PRERTRU,G16TRECPRO,G16TREDHAN,G16TRERDAM,G16USSAFON,G16USSDSNO,G16USSIBAR,G16USSRLEE
0,1-Beaver 1,17.0,48.0,18.0,507.0,27.0,69.0,481.0,13.0,53.0,9.0,525.0,46.0,83.0,1.0,6.0,6.0,469.0,36.0,80.0,454.0,12.0,48.0,3.0,530.0
1,1-Beaver 2,14.0,48.0,25.0,363.0,21.0,72.0,345.0,4.0,51.0,7.0,395.0,38.0,74.0,4.0,4.0,8.0,334.0,19.0,90.0,329.0,8.0,48.0,4.0,390.0
2,1-Beaver 3,11.0,31.0,6.0,289.0,14.0,44.0,269.0,10.0,22.0,6.0,302.0,22.0,49.0,0.0,3.0,4.0,268.0,18.0,52.0,254.0,8.0,30.0,2.0,300.0
3,1-Beaver 4,3.0,16.0,9.0,100.0,12.0,23.0,92.0,0.0,15.0,4.0,114.0,10.0,23.0,0.0,3.0,0.0,101.0,14.0,19.0,94.0,3.0,7.0,4.0,116.0
4,1-Greenville,0.0,12.0,0.0,74.0,0.0,16.0,67.0,1.0,14.0,2.0,72.0,10.0,4.0,2.0,0.0,0.0,76.0,5.0,20.0,57.0,0.0,13.0,0.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,9-Henrieville,1.0,11.0,10.0,94.0,9.0,20.0,87.0,0.0,16.0,5.0,96.0,15.0,27.0,0.0,2.0,0.0,75.0,15.0,21.0,81.0,4.0,13.0,0.0,101.0
2221,9-Pang1,14.0,45.0,18.0,430.0,28.0,62.0,407.0,7.0,41.0,7.0,468.0,50.0,67.0,1.0,11.0,5.0,391.0,22.0,68.0,409.0,4.0,49.0,2.0,464.0
2222,9-Pang2,4.0,33.0,9.0,343.0,10.0,48.0,323.0,7.0,37.0,6.0,348.0,35.0,53.0,0.0,7.0,4.0,302.0,23.0,41.0,320.0,5.0,31.0,1.0,357.0
2223,9-Ticaboo,0.0,4.0,7.0,12.0,4.0,5.0,14.0,2.0,3.0,2.0,17.0,3.0,5.0,0.0,0.0,2.0,14.0,6.0,4.0,13.0,0.0,7.0,2.0,15.0


,joinid,G16ATGAISB,G16ATGDHAR,G16ATGLMCC,G16ATGRREY,G16AUDAGRE,G16AUDDMIT,G16AUDRDOU,G16GOVASCH,G16GOVDWEI,G16GOVLKAM,G16GOVRHER,G16PREDCLI,G16PREIMCM,G16PREISTE,G16PRELJOH,G16PREOOTH,G16PRERTRU,G16TRECPRO,G16TREDHAN,G16TRERDAM,G16USSAFON,G16USSDSNO,G16USSIBAR,G16USSRLEE
2167,7-AL11,18.0,30.0,38.0,504.0,26.0,60.0,498.0,14.0,46.0,24.0,516.0,28.0,50.0,0.0,8.0,14.0,502.0,42.0,62.0,474.0,8.0,34.0,8.0,546.0


In [50]:
ut_vest['joinid'] = ut_vest['CountyID'].astype(str)+'-'+ut_vest['precinct']

In [51]:
display(ut_vest)

,CountyID,PrecinctID,county,precinct,G16PRERTRU,G16PREDCLI,G16PRELJOH,G16PREISTE,G16PREIMCM,G16PREOOTH,G16USSRLEE,G16USSDSNO,G16USSAFON,G16USSIBAR,G16GOVRHER,G16GOVDWEI,G16GOVLKAM,G16GOVASCH,G16ATGRREY,G16ATGDHAR,G16ATGLMCC,G16ATGAISB,G16AUDRDOU,G16AUDDMIT,G16AUDAGRE,G16TRERDAM,G16TREDHAN,G16TRECPRO,geometry,joinid
0,1,BV01,Beaver,Beaver 1,469,46,6,1,83,3,530,48,12,3,525,53,9,13,507,48,18,17,481,69,27,454,80,36,"POLYGON ((358362.560 4238423.840, 358372.350 4...",1-Beaver 1
1,1,BV02,Beaver,Beaver 2,334,38,4,4,74,4,390,48,8,4,395,51,7,4,363,48,25,14,345,72,21,329,90,19,"POLYGON ((355866.750 4240506.080, 356860.230 4...",1-Beaver 2
2,1,BV03,Beaver,Beaver 3,268,22,3,0,49,2,300,30,8,2,302,22,6,10,289,31,6,11,269,44,14,254,52,18,"POLYGON ((358902.400 4261055.000, 358909.700 4...",1-Beaver 3
3,1,BV04,Beaver,Beaver 4,101,10,3,0,23,0,116,7,3,4,114,15,4,0,100,16,9,3,92,23,12,94,19,14,"POLYGON ((367866.500 4262536.100, 367842.500 4...",1-Beaver 4
4,1,GR01,Beaver,Greenville,76,10,0,2,4,0,77,13,0,0,72,14,2,1,74,12,0,0,67,16,0,57,20,5,"POLYGON ((348977.460 4223848.220, 348973.090 4...",1-Greenville
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2291,7,RO81,Duchesne,RO81,216,9,2,1,18,0,223,18,3,0,207,18,16,5,215,10,15,3,216,16,8,198,14,28,"POLYGON ((580836.220 4464060.720, 580837.220 4...",7-RO81
2292,7,RO82,Duchesne,RO82,75,15,2,0,13,1,87,13,3,1,80,18,4,3,80,16,6,0,72,24,4,70,21,10,"POLYGON ((586967.910 4461063.280, 586317.330 4...",7-RO82
2293,7,RO91,Duchesne,RO91,0,0,0,0,2,1,3,0,0,0,3,0,0,0,3,0,0,0,3,0,0,2,0,1,"POLYGON ((583252.560 4462118.640, 583274.150 4...",7-RO91
2294,7,TA11,Duchesne,TA11,197,12,3,0,17,0,203,13,3,7,195,25,2,6,193,17,4,6,175,30,11,173,26,17,"POLYGON ((533169.000 4511919.160, 533169.000 4...",7-TA11


In [53]:
join_attempt_one = pd.merge(ut_vest,pivoted_df,how="outer",on="joinid",indicator=True)
print(join_attempt_one["_merge"].value_counts())

right_only = join_attempt_one[join_attempt_one["_merge"]=="right_only"]
left_only = join_attempt_one[join_attempt_one["_merge"]=="left_only"]
merged = join_attempt_one[join_attempt_one["_merge"]=="both"]

left_only.to_csv("./only_vest.csv")
right_only.to_csv("./only_source.csv")
merged.to_csv("./merged_votes.csv")

both          2219
left_only       77
right_only       6
Name: _merge, dtype: int64


In [59]:
name_change_dict = {
    '2-Tremonton 04': '2-Trementon 04'
}

In [60]:
pivoted_df["joinid"] = pivoted_df["joinid"].map(name_change_dict).fillna(pivoted_df["joinid"])

In [62]:
join_attempt_two = pd.merge(ut_vest,pivoted_df,how="outer",on="joinid",indicator=True)
print(join_attempt_two["_merge"].value_counts())

right_only = join_attempt_two[join_attempt_two["_merge"]=="right_only"]
left_only = join_attempt_two[join_attempt_two["_merge"]=="left_only"]
merged = join_attempt_two[join_attempt_two["_merge"]=="both"]

left_only.to_csv("./only_vest.csv")
right_only.to_csv("./only_source.csv")
merged.to_csv("./merged_votes.csv")

both          2220
left_only       76
right_only       5
Name: _merge, dtype: int64


In [63]:
races = [
    'G16ATGAISB', 'G16ATGDHAR', 'G16ATGLMCC',
    'G16ATGRREY', 'G16AUDAGRE', 'G16AUDDMIT', 'G16AUDRDOU',
    'G16GOVASCH', 'G16GOVDWEI', 'G16GOVLKAM', 'G16GOVRHER',
    'G16PREDCLI', 'G16PREIMCM', 'G16PREISTE', 'G16PRELJOH',
    'G16PREOOTH', 'G16PRERTRU', 'G16TRECPRO', 'G16TREDHAN',
    'G16TRERDAM', 'G16USSAFON', 'G16USSDSNO', 'G16USSIBAR', 
    'G16USSRLEE']


In [71]:
zero_votes = True
for index, row in left_only.iterrows():
    for race in races:
        if row[(race + '_x')] != 0:
            print(row, race)
            zero_votes = False
if zero_votes == True:
    print('No non-zero vote precincts in VEST file')

No non-zero vote precincts in VEST file


In [75]:
zero_votes = True
for index, row in right_only.iterrows():
    for race in races:
        if row[(race + '_y')] != 0:
            print('non zero race in precinct', row['joinid'])
            print('race name is: ', race)
            print('vote count is: ', row[(race + '_y')])
            zero_votes = False
    print('---------------')
if zero_votes == True:
    print('No non-zero vote precincts in VEST file')

---------------
non zero race in precinct 18-Election Total
race name is:  G16GOVASCH
vote count is:  5094.0
non zero race in precinct 18-Election Total
race name is:  G16GOVDWEI
vote count is:  181462.0
non zero race in precinct 18-Election Total
race name is:  G16GOVLKAM
vote count is:  11671.0
non zero race in precinct 18-Election Total
race name is:  G16GOVRHER
vote count is:  218570.0
---------------
---------------
non zero race in precinct 25-CH02:UN
race name is:  G16ATGRREY
vote count is:  1.0
non zero race in precinct 25-CH02:UN
race name is:  G16AUDRDOU
vote count is:  1.0
non zero race in precinct 25-CH02:UN
race name is:  G16GOVRHER
vote count is:  1.0
non zero race in precinct 25-CH02:UN
race name is:  G16PREIMCM
vote count is:  1.0
non zero race in precinct 25-CH02:UN
race name is:  G16TRERDAM
vote count is:  1.0
non zero race in precinct 25-CH02:UN
race name is:  G16USSRLEE
vote count is:  1.0
---------------
non zero race in precinct 8-Canvas
race name is:  G16ATGAISB


In [76]:
votes_df = merged
county_nums = list(range(1,30))
total_diff = 0
for county in county_nums:
    for race in races:
        vest_votes = votes_df.loc[votes_df['CountyID'] == county, (race + '_x')].sum()
        source_votes = votes_df.loc[votes_df['CountyID'] == county, (race + '_y')].astype(int).sum()
        diff = abs(vest_votes-source_votes)
        #print('In County number', county, 'for race', race, 'the VEST total is', vest_votes, 'and source total is', source_votes, 'and their difference is', abs(vest_votes-source_votes))
        if diff > 0:
            total_diff += diff
            print('County number:', county)
            print('Race code:', race)
            print('VEST total:', vest_votes)
            print('Source total:', source_votes)
            print('Difference of:', diff)
            print('----------------------------')
print('total difference between two files is:', total_diff)

County number: 1
Race code: G16PREOOTH
VEST total: 20.0
Source total: 34
Difference of: 14.0
----------------------------
County number: 2
Race code: G16PREOOTH
VEST total: 299.0
Source total: 442
Difference of: 143.0
----------------------------
County number: 3
Race code: G16PREOOTH
VEST total: 799.0
Source total: 1361
Difference of: 562.0
----------------------------
County number: 4
Race code: G16PREOOTH
VEST total: 113.0
Source total: 160
Difference of: 47.0
----------------------------
County number: 4
Race code: G16USSRLEE
VEST total: 5607.0
Source total: 5606
Difference of: 1.0
----------------------------
County number: 5
Race code: G16PREOOTH
VEST total: 7.0
Source total: 8
Difference of: 1.0
----------------------------
County number: 6
Race code: G16AUDAGRE
VEST total: 8722.0
Source total: 0
Difference of: 8722.0
----------------------------
County number: 6
Race code: G16PREOOTH
VEST total: 1408.0
Source total: 3296
Difference of: 1888.0
----------------------------
County

In [87]:
ut_geo_copy = ut_geo.copy()
ut_geo_copy['joinid'] = ut_geo_copy['CountyID'].astype(str) + '-' + ut_geo_copy['PrecinctID']
dup = ut_geo_copy['joinid'].value_counts()
dup_list = (dup[dup>1]).index
print(dup_list)
print(len(ut_geo_copy.joinid.unique()))
display(ut_geo_copy)

Index(['26-54', '29-WNO003', '29-WHV004', '5-1002', '10-09', '26-32', '10-04',
       '3-LOG27', '29-WHV002', '6-WP02',
       ...
       '26-53', '18-WJD055', '23-T-3', '18-COT036', '18-MID002', '18-SSL003',
       '2-ELWD', '23-ERD-12', '25-SR03', '23-STK-02'],
      dtype='object', length=571)
2224


,OBJECTID,CountyID,VistaID,PrecinctID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area,geometry,joinid
0,1,13,BW,BW,None,None,2012-01-01,None,kanerds,2012-02-02,{886D020D-51B9-48B5-A28D-CC764A5E0F82},0.195803,0.001577,"POLYGON ((-111.65149 37.09008, -111.65157 37.0...",13-BW
1,2,25,OR20,OR20,None,2021.1,2021-03-02,None,AndreaB,2021-03-02,{FFE08944-1D79-42D7-962E-AC71AD650996},0.044090,0.000095,"POLYGON ((-111.70145 40.31183, -111.70274 40.3...",25-OR20
2,3,18,SLC117:00,SLC117,00,None,2019-12-16,Salt Lake City,None,2019-12-16,{43BB406D-2A87-417B-81D7-C34BCDDD17DA},0.039456,0.000041,"POLYGON ((-111.83496 40.74981, -111.83440 40.7...",18-SLC117
3,4,27,COHV28,COHV28,None,None,2012-01-01,None,nancy.lucchetti,2019-02-25,{9B6E24D8-8475-4BF1-BFA5-702BE7C1A5E4},2.571008,0.063659,"POLYGON ((-113.26559 37.19438, -113.26488 37.1...",27-COHV28
4,5,6,KA09:I-C-,KA09,I-C-,2013.12,None,None,GISMASTER,2021-02-10,{D45AF05A-8177-4286-9CEF-758E587CFA5D},0.039713,0.000036,"POLYGON ((-111.91493 41.04574, -111.91488 41.0...",6-KA09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061,3062,6,WX01:U-S-,WX01,U-S-,2015.12,None,None,GISMASTER,2021-02-10,{9FE7FFD2-572E-4524-89F4-170C424E08EB},0.081443,0.000252,"POLYGON ((-111.93003 40.88782, -111.93003 40.8...",6-WX01
3062,3063,18,TAY041:00,TAY041,00,None,2019-12-16,Taylorsville,None,2019-12-16,{A7FFD04B-9AFA-42A9-BD52-6E6617DFEFCF},0.036616,0.000062,"POLYGON ((-111.93201 40.64322, -111.93195 40.6...",18-TAY041
3063,3064,18,MIL032:01,MIL032,01,None,2019-12-16,Millcreek,None,2019-12-16,{B0B47492-516F-4CB7-97E6-C2D3069FD2FA},0.060945,0.000100,"POLYGON ((-111.81371 40.71131, -111.81371 40.7...",18-MIL032
3064,3065,18,COT038:00,COT038,00,None,2019-12-16,Cottonwood Heights,None,2019-12-16,{3F65B9EC-984A-4B2F-B34B-E8337B060AF0},0.045916,0.000078,"POLYGON ((-111.78793 40.58628, -111.78788 40.5...",18-COT038


In [91]:
ut_geo_valid = ut_geo_copy.loc[ut_geo_copy.is_valid]

display(ut_geo_copy[ut_geo_copy['PrecinctID'].isnull()])

combo_precinct = ut_geo_valid.dissolve(by='PrecinctID')

display(combo_precinct)

,OBJECTID,CountyID,VistaID,PrecinctID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area,geometry,joinid


,geometry,OBJECTID,CountyID,VistaID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area,joinid
PrecinctID,,,,,,,,,,,,,,
01,"POLYGON ((-109.54289 38.55717, -109.54288 38.5...",157,10,01:570,570,GC2,2015-09-08,None,None,2015-09-08,{16F98265-D962-4908-BA22-C6513009938F},0.002567,2.430688e-07,10-01
02,"POLYGON ((-109.52433 38.56376, -109.52431 38.5...",189,10,02:470,470,GC2,2015-09-08,None,None,2015-09-08,{7FC57E66-FB6F-4D4F-9B33-0B83942D198E},0.023046,2.306059e-05,10-02
03,"POLYGON ((-109.57966 38.57209, -109.57978 38.5...",541,10,03:470,470,GC2,2015-09-08,None,None,2015-09-08,{03386394-2FCF-48A5-A7E5-3E68443F4CFF},0.034036,7.101122e-05,10-03
04,"POLYGON ((-109.57130 38.57938, -109.57318 38.5...",210,10,04:570,570,GC2,2015-09-08,None,None,2015-09-08,{A2FBF720-3942-4F2B-9F73-0B74E280E321},1.810748,6.914687e-02,10-04
05,"POLYGON ((-109.52951 38.56251, -109.53018 38.5...",593,10,05:570,570,GC2,2015-09-08,None,None,2015-09-08,{0E7929BD-ABCC-419C-AD0E-A431667A112F},0.040510,7.670614e-05,10-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WX04,"POLYGON ((-111.89276 40.86605, -111.89293 40.8...",1649,6,WX04:I-S-,I-S-,2013.12,None,None,GISMASTER,2021-02-10,{447F579D-96F8-4D49-B305-1D540AB5C891},0.063161,1.179121e-04,6-WX04
WX05,"POLYGON ((-111.91189 40.86173, -111.91189 40.8...",1858,6,WX05:I-S-,I-S-,2013.12,None,None,GISMASTER,2021-02-10,{619CFE38-72DF-4155-B4EC-D7745E4DC9EE},0.055936,1.190062e-04,6-WX05
WX06,"POLYGON ((-111.96421 40.86148, -111.96421 40.8...",1328,6,WX06:U-S-,U-S-,2013.12,None,None,GISMASTER,2021-02-10,{324C549D-16C6-4E32-9CEE-6FB8DE7102E7},0.070044,1.606149e-04,6-WX06


In [95]:
election_data = merged.drop(columns = '_merge')
join_attempt_geo = pd.merge(election_data, combo_precinct, how="outer",on="joinid",indicator=True)
print(join_attempt_geo["_merge"].value_counts())


merged_geo = join_attempt_geo[join_attempt_geo["_merge"]=="both"]
right_only_geo = join_attempt_geo[join_attempt_geo["_merge"]=="right_only"]
right_only_geo_sorted = right_only_geo.sort_values('joinid')
left_only_geo = join_attempt_geo[join_attempt_geo["_merge"]=="left_only"]


left_only_geo.to_csv("./only_vest_geo.csv")
right_only_geo_sorted.to_csv("./only_source_geo.csv")
merged_geo.to_csv('./both_geo.csv')

both          1306
left_only      914
right_only     828
Name: _merge, dtype: int64


In [108]:
left_only_geo_copy = left_only_geo.drop(columns = [
    '_merge', 'geometry_y', 'CountyID_y', 'VistaID', 'SubPrecinc', 
    'VersionNbr', 'EffectiveD', 'AliasName', 'Comments', 'RcvdDate', 
    'GlobalID', 'SHAPE_Leng', 'SHAPE_Area', 'OBJECTID'])
left_only_geo_copy['joinid'] = left_only_geo_copy['CountyID_x'].astype(int).astype(str) + '-' + left_only_geo_copy['PrecinctID'].astype(str)

In [109]:
display(left_only_geo_copy)

,CountyID_x,PrecinctID,county,precinct,G16PRERTRU_x,G16PREDCLI_x,G16PRELJOH_x,G16PREISTE_x,G16PREIMCM_x,G16PREOOTH_x,G16USSRLEE_x,G16USSDSNO_x,G16USSAFON_x,G16USSIBAR_x,G16GOVRHER_x,G16GOVDWEI_x,G16GOVLKAM_x,G16GOVASCH_x,G16ATGRREY_x,G16ATGDHAR_x,G16ATGLMCC_x,G16ATGAISB_x,G16AUDRDOU_x,G16AUDDMIT_x,G16AUDAGRE_x,G16TRERDAM_x,G16TREDHAN_x,G16TRECPRO_x,geometry_x,joinid,G16ATGAISB_y,G16ATGDHAR_y,G16ATGLMCC_y,G16ATGRREY_y,G16AUDAGRE_y,G16AUDDMIT_y,G16AUDRDOU_y,G16GOVASCH_y,G16GOVDWEI_y,G16GOVLKAM_y,G16GOVRHER_y,G16PREDCLI_y,G16PREIMCM_y,G16PREISTE_y,G16PRELJOH_y,G16PREOOTH_y,G16PRERTRU_y,G16TRECPRO_y,G16TREDHAN_y,G16TRERDAM_y,G16USSAFON_y,G16USSDSNO_y,G16USSIBAR_y,G16USSRLEE_y
0,1.0,BV01,Beaver,Beaver 1,469.0,46.0,6.0,1.0,83.0,3.0,530.0,48.0,12.0,3.0,525.0,53.0,9.0,13.0,507.0,48.0,18.0,17.0,481.0,69.0,27.0,454.0,80.0,36.0,"POLYGON ((358362.560 4238423.840, 358372.350 4...",1-BV01,17.0,48.0,18.0,507.0,27.0,69.0,481.0,13.0,53.0,9.0,525.0,46.0,83.0,1.0,6.0,6.0,469.0,36.0,80.0,454.0,12.0,48.0,3.0,530.0
1,1.0,BV02,Beaver,Beaver 2,334.0,38.0,4.0,4.0,74.0,4.0,390.0,48.0,8.0,4.0,395.0,51.0,7.0,4.0,363.0,48.0,25.0,14.0,345.0,72.0,21.0,329.0,90.0,19.0,"POLYGON ((355866.750 4240506.080, 356860.230 4...",1-BV02,14.0,48.0,25.0,363.0,21.0,72.0,345.0,4.0,51.0,7.0,395.0,38.0,74.0,4.0,4.0,8.0,334.0,19.0,90.0,329.0,8.0,48.0,4.0,390.0
2,1.0,BV03,Beaver,Beaver 3,268.0,22.0,3.0,0.0,49.0,2.0,300.0,30.0,8.0,2.0,302.0,22.0,6.0,10.0,289.0,31.0,6.0,11.0,269.0,44.0,14.0,254.0,52.0,18.0,"POLYGON ((358902.400 4261055.000, 358909.700 4...",1-BV03,11.0,31.0,6.0,289.0,14.0,44.0,269.0,10.0,22.0,6.0,302.0,22.0,49.0,0.0,3.0,4.0,268.0,18.0,52.0,254.0,8.0,30.0,2.0,300.0
3,1.0,BV04,Beaver,Beaver 4,101.0,10.0,3.0,0.0,23.0,0.0,116.0,7.0,3.0,4.0,114.0,15.0,4.0,0.0,100.0,16.0,9.0,3.0,92.0,23.0,12.0,94.0,19.0,14.0,"POLYGON ((367866.500 4262536.100, 367842.500 4...",1-BV04,3.0,16.0,9.0,100.0,12.0,23.0,92.0,0.0,15.0,4.0,114.0,10.0,23.0,0.0,3.0,0.0,101.0,14.0,19.0,94.0,3.0,7.0,4.0,116.0
4,1.0,GR01,Beaver,Greenville,76.0,10.0,0.0,2.0,4.0,0.0,77.0,13.0,0.0,0.0,72.0,14.0,2.0,1.0,74.0,12.0,0.0,0.0,67.0,16.0,0.0,57.0,20.0,5.0,"POLYGON ((348977.460 4223848.220, 348973.090 4...",1-GR01,0.0,12.0,0.0,74.0,0.0,16.0,67.0,1.0,14.0,2.0,72.0,10.0,4.0,2.0,0.0,0.0,76.0,5.0,20.0,57.0,0.0,13.0,0.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,29.0,WCR001,Weber,WCR001,142.0,39.0,8.0,2.0,44.0,2.0,179.0,50.0,5.0,3.0,178.0,47.0,8.0,4.0,175.0,38.0,14.0,8.0,162.0,52.0,15.0,162.0,52.0,17.0,"MULTIPOLYGON (((409684.030 4557452.630, 409684...",29-WCR001,8.0,38.0,14.0,175.0,15.0,52.0,162.0,4.0,47.0,8.0,178.0,39.0,44.0,2.0,8.0,4.0,142.0,17.0,52.0,162.0,5.0,50.0,3.0,179.0
2167,29.0,WCR002,Weber,WCR002,5.0,11.0,1.0,0.0,2.0,0.0,9.0,11.0,0.0,0.0,10.0,10.0,0.0,0.0,10.0,9.0,1.0,0.0,8.0,11.0,1.0,6.0,13.0,0.0,"MULTIPOLYGON (((411121.260 4559199.700, 411120...",29-WCR002,0.0,9.0,1.0,10.0,1.0,11.0,8.0,0.0,10.0,0.0,10.0,11.0,2.0,0.0,1.0,0.0,5.0,0.0,13.0,6.0,0.0,11.0,0.0,9.0
2168,29.0,WCR003,Weber,WCR003,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,"POLYGON ((412080.060 4560211.250, 412079.080 4...",29-WCR003,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2169,29.0,WCS001,Weber,WCS001,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,"POLYGON ((418624.680 4560352.340, 418624.810 4...",29-WCS001,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0


In [130]:
join_attempt_geo2 = pd.merge(left_only_geo_copy, combo_precinct, how="outer",on="joinid",indicator=True)
print(join_attempt_geo2["_merge"].value_counts())

merged_geo2 = join_attempt_geo2[join_attempt_geo2["_merge"]=="both"]
right_only_geo2 = join_attempt_geo2[join_attempt_geo2["_merge"]=="right_only"]
right_only_geo2_sorted = right_only_geo2.sort_values('joinid')
left_only_geo2 = join_attempt_geo2[join_attempt_geo2["_merge"]=="left_only"]

left_only_geo2.to_csv("./only_vest_geo.csv")
right_only_geo2_sorted.to_csv("./only_source_geo.csv")
merged_geo2.to_csv('./both_geo.csv')

right_only    1506
both           628
left_only      286
Name: _merge, dtype: int64


### Attempt to join the remaining on Vista ID

In [137]:
combo_precinct_vista = ut_geo_copy
combo_precinct_vista['joinid'] = combo_precinct_vista['CountyID'].astype(str) + '-' + combo_precinct_vista['VistaID']
left_only_geo2_copy = left_only_geo2.drop(columns = [
    '_merge', 'VistaID', 'SubPrecinc', 
    'VersionNbr', 'EffectiveD', 'AliasName', 'Comments', 'RcvdDate', 
    'GlobalID', 'SHAPE_Leng', 'SHAPE_Area', 'OBJECTID'])
join_attempt_geo3 = pd.merge(left_only_geo2_copy, combo_precinct_vista, how="outer",on="joinid",indicator=True)
print(join_attempt_geo3["_merge"].value_counts())

merged_geo3 = join_attempt_geo2[join_attempt_geo2["_merge"]=="both"]
right_only_geo3 = join_attempt_geo2[join_attempt_geo2["_merge"]=="right_only"]
right_only_geo3_sorted = right_only_geo2.sort_values('joinid')
left_only_geo3 = join_attempt_geo2[join_attempt_geo2["_merge"]=="left_only"]

left_only_geo2.to_csv("./only_vest_geo2.csv")
right_only_geo2_sorted.to_csv("./only_source_geo2.csv")


right_only    2954
left_only      202
both           112
Name: _merge, dtype: int64


In [132]:
geo_dict = {
    '5-1001:DS1': '5-1001:DS1,SD1',
 '5-1001:SD1': '5-1001:DS1,SD1',
 '5-1001:SD2': '5-1001:DS2,SD2',
 '5-1001:DS2': '5-1001:DS2,SD2',
 '5-1002:MS3': '5-1002:MS3,SD3',
 '6-CF01:I-N-': '6-CF01',
 '7-RO92': '7-R092',
 '10-01:470': '10-01:70',
 '10-01:570': '10-01:70',
 '10-02:569': '10-02:69',
 '10-02:570': '10-02:70',
 '10-02:470': '10-02:70',
 '10-03:570': '10-03:70',
 '10-03:470': '10-03:70',
 '10-04:569': '10-04:69',
 '10-04:470': '10-04:70',
 '10-04:570': '10-04:70',
 '10-05:569': '10-05:69',
 '10-05:470': '10-05:70',
 '10-05:570': '10-05:70',
 '10-06:469': '10-06:69',
 '10-06:569': '10-06:69',
 '10-06:470': '10-06:70',
 '10-06:570': '10-06:70',
 '10-07:569': '10-07:69',
 '10-07:570': '10-07:70',
 '10-08:469': '10-08:69',
 '10-08:569': '10-08:69',
 '10-09:569': '10-09:69',
 '10-09:470': '10-09:70',
 '10-09:570': '10-09:70',
 '10-10:470': '10-10:70',
 '10-10:570': '10-10:70',
 '10-11:469': '10-11:69',
 '10-11:569': '10-11:69',
 '10-11:470': '10-11:70',
 '10-11:570': '10-11:70',
 '15-10:2': '15-10:4',
 '15-11:2': '15-11:4',
 '15-12:2': '15-12:4',
 '15-13:2': '15-13:5',
 '15-14:3': '15-14:5',
 '15-15:2': '15-15:5',
 '15-4:1': '15-4:2',
 '15-5:3': '15-5:2',
 '15-6:3': '15-6:2',
 '18-RIV001:00': '18-RIV001',
 '18-RIV004:00': '18-RIV004',
 '18-RIV005:00': '18-RIV005',
 '18-TAY001:01': '18-TAY001',
 '18-TAY001:90': '18-TAY001',
 '18-TAY002:00': '18-TAY002',
 '21-1:1': '21-1',
 '21-1:2': '21-1',
 '21-10:1': '21-10',
 '21-10:2': '21-10',
 '21-11:2': '21-11.2',
 '21-12:1': '21-12',
 '21-2:1': '21-2',
 '21-2:2': '21-2',
 '21-20:2': '21-20.2',
 '21-21:1': '21-21',
 '21-23:2': '21-23.2',
 '21-24:2': '21-24.2',
 '21-4:1': '21-4',
 '21-4:2': '21-4',
 '21-5:1': '21-5',
 '21-5:2': '21-5',
 '21-6:1': '21-6',
 '21-6:2': '21-6',
 '21-7:1': '21-7.1.3',
 '21-7:3': '21-7.1.3',
 '21-7:2': '21-7.2',
 '21-8:1': '21-8',
 '21-8:2': '21-8',
 '21-9:1': '21-9',
 '21-9:2': '21-9',
 '23-RV-01:1': '23-RV-01:1-01:2',
 '23-RV-01:2': '23-RV-01:1-01:2',
 '23-STK-01:1': '23-STK-01:1-01:2',
 '23-STK-01:2': '23-STK-01:1-01:2',
 '23-STK-02:1': '23-STK-02:1-02:2',
 '23-STK-02:2': '23-STK-02:1-02:2',
 '23-T-03:1': '23-T-03:1-03:2',
 '23-T-03:2': '23-T-03:1-03:2',
 '23-T-04:1': '23-T-04:1-04:2',
 '23-T-04:2': '23-T-04:1-04:2',
 '23-VRN-01:1': '23-VRN-01:1-01:2',
 '23-VRN-01:2': '23-VRN-01:1-01:2',
 '8-9:20': '8-9',
 '8-9:21': '8-9',
 '8-3:5': '8-3',
 '8-3:6': '8-3',
 '8-11:24': '8-11',
 '8-11:25': '8-11',
 '8-5:10': '8-5',
 '8-5:9': '8-5',
 '8-7:14': '8-7',
 '8-7:15': '8-7',
 '8-6:11': '8-6',
 '8-6:13': '8-6',
 '8-1:1': '8-1',
 '8-1:2': '8-1',
 '8-12:26': '8-12',
 '8-12:27': '8-12',
 '8-13:28': '8-13',
 '8-13:29': '8-13'
           }

In [134]:
combo_precinct_vista_copy = combo_precinct_vista
combo_precinct_vista_copy["joinid"] = combo_precinct_vista_copy["joinid"].map(geo_dict).fillna(combo_precinct_vista_copy["joinid"])
display(combo_precinct_vista_copy)
combo_precinct_vista_copy = combo_precinct_vista_copy.loc[combo_precinct_vista_copy.is_valid]
dissolved_geo = combo_precinct_vista_copy.dissolve(by='joinid')
display(dissolved_geo)

,OBJECTID,CountyID,VistaID,PrecinctID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area,geometry,joinid
0,1,13,BW,BW,None,None,2012-01-01,None,kanerds,2012-02-02,{886D020D-51B9-48B5-A28D-CC764A5E0F82},0.195803,0.001577,"POLYGON ((-111.65149 37.09008, -111.65157 37.0...",13-BW
1,2,25,OR20,OR20,None,2021.1,2021-03-02,None,AndreaB,2021-03-02,{FFE08944-1D79-42D7-962E-AC71AD650996},0.044090,0.000095,"POLYGON ((-111.70145 40.31183, -111.70274 40.3...",25-OR20
2,3,18,SLC117:00,SLC117,00,None,2019-12-16,Salt Lake City,None,2019-12-16,{43BB406D-2A87-417B-81D7-C34BCDDD17DA},0.039456,0.000041,"POLYGON ((-111.83496 40.74981, -111.83440 40.7...",18-SLC117:00
3,4,27,COHV28,COHV28,None,None,2012-01-01,None,nancy.lucchetti,2019-02-25,{9B6E24D8-8475-4BF1-BFA5-702BE7C1A5E4},2.571008,0.063659,"POLYGON ((-113.26559 37.19438, -113.26488 37.1...",27-COHV28
4,5,6,KA09:I-C-,KA09,I-C-,2013.12,None,None,GISMASTER,2021-02-10,{D45AF05A-8177-4286-9CEF-758E587CFA5D},0.039713,0.000036,"POLYGON ((-111.91493 41.04574, -111.91488 41.0...",6-KA09:I-C-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061,3062,6,WX01:U-S-,WX01,U-S-,2015.12,None,None,GISMASTER,2021-02-10,{9FE7FFD2-572E-4524-89F4-170C424E08EB},0.081443,0.000252,"POLYGON ((-111.93003 40.88782, -111.93003 40.8...",6-WX01:U-S-
3062,3063,18,TAY041:00,TAY041,00,None,2019-12-16,Taylorsville,None,2019-12-16,{A7FFD04B-9AFA-42A9-BD52-6E6617DFEFCF},0.036616,0.000062,"POLYGON ((-111.93201 40.64322, -111.93195 40.6...",18-TAY041:00
3063,3064,18,MIL032:01,MIL032,01,None,2019-12-16,Millcreek,None,2019-12-16,{B0B47492-516F-4CB7-97E6-C2D3069FD2FA},0.060945,0.000100,"POLYGON ((-111.81371 40.71131, -111.81371 40.7...",18-MIL032:01
3064,3065,18,COT038:00,COT038,00,None,2019-12-16,Cottonwood Heights,None,2019-12-16,{3F65B9EC-984A-4B2F-B34B-E8337B060AF0},0.045916,0.000078,"POLYGON ((-111.78793 40.58628, -111.78788 40.5...",18-COT038:00


,geometry,OBJECTID,CountyID,VistaID,PrecinctID,SubPrecinc,VersionNbr,EffectiveD,AliasName,Comments,RcvdDate,GlobalID,SHAPE_Leng,SHAPE_Area
joinid,,,,,,,,,,,,,,
1-BV01:1,"POLYGON ((-112.61943 38.28272, -112.61932 38.2...",127,1,BV01:1,BV01,1,2013-1,2012-01-01,None,None,2013-10-16,{DFEFBA5B-6190-45B6-BDB7-83C32DA1B1E9},0.078745,0.000202
1-BV02:1,"POLYGON ((-112.64838 38.30108, -112.63701 38.3...",1067,1,BV02:1,BV02,1,2013-1,2012-01-01,None,None,2013-10-16,{EBA701A3-E894-4DB7-88F5-A663A20C8579},0.226984,0.000838
1-BV03:1,"MULTIPOLYGON (((-112.65342 38.22070, -112.6533...",1021,1,BV03:1,BV03,1,2013-1,2013-10-16,None,None,2013-10-16,{8054F647-FDAE-4386-82EC-7627DB7A718D},0.323054,0.000728
1-BV03:2,"MULTIPOLYGON (((-112.62208 38.31248, -112.6215...",416,1,BV03:2,BV03,2,2013-1,2013-10-16,None,None,2013-10-16,{54422F66-0177-4C63-846B-E5D18BDEC381},2.428363,0.077798
1-BV04:1,"POLYGON ((-112.51533 38.50140, -112.51560 38.5...",2978,1,BV04:1,BV04,1,2013-1,2012-01-01,None,None,2013-10-16,{AFCD9F5F-FFE0-4556-8948-1DE063F45DD0},1.435445,0.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9-PA8:1,"POLYGON ((-112.44624 37.81799, -112.45084 37.8...",862,9,PA8:1,PA8,None,None,None,None,initial estimate by AGRC,2011-07-12,{A38544CA-669B-4A6E-A631-E1C1EC547A52},0.118368,0.000351
9-PA8:2,"POLYGON ((-112.44624 37.81799, -112.44272 37.8...",915,9,PA8:2,PA8,None,None,None,None,initial estimate by AGRC,2011-07-12,{9EF5CDBF-164A-40E3-BC3B-590B3C6928AD},1.826924,0.069706
9-TI9:1,"POLYGON ((-110.20313 37.94548, -110.20329 37.9...",1065,9,TI9:1,TI9,1,None,2012-01-01,None,initial estimate by AGRC,2011-07-12,{8003838D-E71D-44EA-BD8A-0B6B5C5666F1},3.416384,0.407979


In [136]:
left_only_geo3_copy = left_only_geo3.drop(columns = [
    '_merge', 'VistaID', 'SubPrecinc', 
    'VersionNbr', 'EffectiveD', 'AliasName', 'Comments', 'RcvdDate', 
    'GlobalID', 'SHAPE_Leng', 'SHAPE_Area', 'OBJECTID'])
join_attempt_geo4 = pd.merge(left_only_geo3_copy, dissolved_geo, how="outer",on="joinid",indicator=True)
print(join_attempt_geo4["_merge"].value_counts())

merged_geo3 = join_attempt_geo3[join_attempt_geo3["_merge"]=="both"]
right_only_geo3 = join_attempt_geo3[join_attempt_geo3["_merge"]=="right_only"]
right_only_geo3_sorted = right_only_geo3.sort_values('joinid')
left_only_geo3 = join_attempt_geo3[join_attempt_geo3["_merge"]=="left_only"]

#left_only_geo3.to_csv("./only_vest_geo3.csv")
#right_only_geo3_sorted.to_csv("./only_source_geo2.csv")



right_only    2847
left_only      165
both            37
Name: _merge, dtype: int64
